In [ ]:
%load_ext nb_black

Welcome to this lab session 3 on **Time series modeling for air pollution monitoring with a focus on the
calibration of low-cost sensors.**

This lab session is based on the data and methods provided in the study by [Ellen M. Considine et al](https://www.sciencedirect.com/science/article/pii/S0269749120365222).


In the notebook, we will focus on using a regression models (linear model and random forest) to correct the readings of low-cost (CS) sensors.

We will start by presenting the use of a linear model for this purpose and we will explore different combinations of predictor variables in our model and compare the results. The idea here is to simulate a case of invastigating the best model and the features with which we get this model. 

In addition to a linear model, in our reference literature, a random forest model was also used. The follow-up task you will get in this session is to apply a random forest model and observe the results.

# First, lets import the libraries we will be using

In [ ]:
import math
from typing import List, Optional

import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Load the data

In [ ]:
data_root = "./data/"
training_data_path = data_root + "cleaned_training.csv"
test_data_path = data_root + "cleaned_test.csv"

In [ ]:
training_data = pd.read_csv(training_data_path)
test_data = pd.read_csv(test_data_path)

In [ ]:
training_data.head()

# Utility functions

In [ ]:
# More evaluation metrics can be added to the function
def evaluate_model(y, y_hat):
    evaluation = {"RMSE": round(mean_squared_error(y, y_hat, squared=False), 2)}
    return evaluation


# To disaggregate the error metrics to the different locations
def get_disaggregated_metrics(data: "dataframe", y_hat: Optional[List] = None):

    """
    Disaggregate evaluation metrics into locations
    """
    results = {}

    for sensor in data["cs_sensor"].unique():
        data_sensor_idxs = data[data["cs_sensor"] == sensor].index
        true_data = data[["pm_airnow"]][data.index.isin(data_sensor_idxs)]
        pred_data = (
            data[["pm_cs"]][data.index.isin(data_sensor_idxs)]
            if y_hat is None
            else y_hat[data_sensor_idxs]
        )

        eval_metrics = evaluate_model(true_data, pred_data)
        results[sensor] = eval_metrics

    return {"RMSE": results}


def run_training_eval(
    features: dict,
    train: "dataframe" = training_data,
    test: "dataframe" = test_data,
    estimator: "sklearn model" = LinearRegression(),
):

    """
    Run training and evaluation on the data

    Purpose: fast experiment iteration
    """
    y_train = train[["pm_airnow"]]
    X_train = training_data[features]

    y_test = test[["pm_airnow"]]
    X_test = test[features]
    
    model = estimator
    model.fit(X_train, y_train)
    y_hat_test = model.predict(X_test)
    y_hat_train = model.predict(X_train)  # predicted value

    train_eval = evaluate_model(y_train, y_hat_train)
    test_eval = evaluate_model(y_test, y_hat_test)

    return {
        "train_eval": train_eval,
        "test_eval": test_eval,
        "model": model,
        "y_hat_train": y_hat_train,
        "y_hat_test": y_hat_test,
    }

# How do we know we have a useful model? We need a baseline

Before we proceed into deriving correction models, we need to decide how we intend to evaluate our models. 

Our base study applied the coefficient of determination ($R^2$) and root-mean squared error (RMSE). 

RMSE shows us the mean distance between the predicted and actual values.

For this notebook, we will be using RMSE measure which has been defined as part of our utility functions. Lower RMSE values indicate more accurate models.

We will obtain our baseline RMSE by computing the training and test RMSE between `pm_airnow` and `pm_cs`. Any model that will be useful whatsoever should do significantly better (i.e lower) than this baseline.

In [ ]:
baseline_train_rmse = evaluate_model(training_data["pm_airnow"], training_data["pm_cs"])
baseline_test_rmse = evaluate_model(test_data["pm_airnow"], test_data["pm_cs"])

In [ ]:
print(f"Baseline training evaluation is: {baseline_train_rmse}")
print(f"Baseline test evaluation is: {baseline_test_rmse}")

It is clear to us now that any model that learns something from our data should produce less than `5.51` and `7.08`RMSE over the training and test set, respectively. Please note that RMSE is on the scale of our data, meaning it is measured in $µg/m^3$.

# Can we disaggregate these errors to the different sensors/locations?

Starting from the baseline

In [ ]:
baseline_disagg_train_metrics = get_disaggregated_metrics(training_data)

In [ ]:
print("Disaggregated RMSE score")
baseline_disagg_train_metrics["RMSE"]

In [ ]:
baseline_disagg_test_metrics = get_disaggregated_metrics(test_data)

In [ ]:
print("Disaggregated test RMSE")
baseline_disagg_test_metrics["RMSE"]

Again, any model that will be generally useful should do less than these errors in each of these locations (training and test)

# Obtaining our models

**model_1**: CS PM2.5 <br>
**model_2**: CS PM2.5, temperature, humidity <br>
**model_3**: CS PM2.5, temprature, humidity, road length <br>
**model_4**: CS PM2.5, temprature, humidity, road length, hour, month <br>
**model_5**: CS PM2.5, temprature, humidity, road length, hour, mont, weekend <br>

In [ ]:
model_features = {
    "model_1": ["pm_cs"],
    "model_2": ["pm_cs", "temp", "humidity"],
    "model_3": ["pm_cs", "temp", "humidity", "a_road_500"],
    "model_4": [
        "pm_cs",
        "temp",
        "humidity",
        "a_road_500",
        "sin_time",
        "cos_time",
        "sin_month",
        "cos_month",
    ],
    "model_5": [
        "pm_cs",
        "temp",
        "humidity",
        "a_road_500",
        "sin_time",
        "cos_time",
        "sin_month",
        "cos_month",
        "weekend",
    ],
}

###### Let us run our first example using only one feature `pm_cs` 

We need to do the following:
- Choose the features that we will be using to fit our model
- Extract `x_train, x_test, y_train, y_test`, as features and reponse variables for  training and test.
- Fit the model
- Predict
- Evaluate: RMSE and scatter plot

In [ ]:
features = model_features["model_1"]

y_train = training_data[["pm_airnow"]]
X_train = training_data[features]

y_test = test_data[["pm_airnow"]]
X_test = test_data[features]

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Obtain the predicted value as y_hat
y_hat_test = model.predict(X_test)
y_hat_train = model.predict(X_train)  # predicted value

In [ ]:
plt.scatter(y_test, y_hat_test, label="Corrected", alpha=0.3)
plt.scatter(y_test, test_data[["pm_cs"]], label="Not corrected", alpha=0.3)
plt.xlabel("Airnow")
plt.ylabel("CS")
plt.title("Test data evaluation plot")
plt.legend()
plt.show()

**We should only have only one model coefficient here, so we can't actually compare features. This step will be useful once we start scaling into more features**

**Now, let us evaluate and compare to baseline**

In [ ]:
model_train_rmse = evaluate_model(y_train, y_hat_train)
model_test_rmse = evaluate_model(y_test, y_hat_test)

In [ ]:
print(
    f"Resulting training evaluation is: {model_train_rmse} compared to baseline which is: {baseline_train_rmse}"
)

In [ ]:
print(
    f"Resulting test evaluation is: {model_test_rmse} compared to baseline which is: {baseline_test_rmse}"
)

Our model reduces our error both in training and test compared to the baseline we obtained initially. This shows that we have already made meaningful progress using a linear model. It is a good start.

Now, let us observe the errors for each location (disaggregated).

In [ ]:
model_disagg_train_metrics = get_disaggregated_metrics(training_data, y_hat_train)
model_disagg_test_metrics = get_disaggregated_metrics(test_data, y_hat_test)

In [ ]:
print("Model training RMSE")
model_disagg_train_metrics["RMSE"]

In [ ]:
print("Model test RMSE")
model_disagg_test_metrics["RMSE"]

Our model reduces our RMSE error with respect to the baseline; we are correcting the low-cost sensor readings with about 50% reduction in error in `i25_denver` and 30% in `CAMP`. Data from these locations have not been seen by our model. There is some progress, right? See table below.

  | Location| Baseline RMSE| Linear model 1 RMSE|
  |---|---|---|
   |**Train**|---|---|
  |NJH | 4.36| 2.28|
  |i25_glo_1|6.67|3.97|
  |i25_glo_2|4.55|3.57|
  |i25_glo_3|5.41|3.65|
  |la_casa|6.06|2.86|
   |**Train**|---|---|
  |CAMP|2.35|1.64|
  |i25_denver|8.04|4.47|
  

# EXERCISE

We need to repeat this process above for other feature sets defined in `model_features`.

For faster iterations, you can use `run_training_eval` function to iterate over features. Take not that the `run_training_eval` function also takes an `èstimator` argument which defaults to `LinearRegressor()`. You can change the value passed to this argument to `RandomForestRegressor` (which has alrready been imported into this notebook). This way, you can try they random forest model.  

Start with exploring all the possible features defined in `model_features` for `LinearRegressor()` before proceeding to `RandomForestRegressor`.

Find out what your best model is and deepen your exploration of this model using things like scatter plots for camparing corrected and not corrected `pm_cs`values. You model output is the corrected `pm_cs`value.

You can also find the feature importances by using `.feature_importances_`on your trained model. If you apply `run_training_eval`to fit your model, you will find the trained model in by using the `model`on the output dictionary.

You can then proceed to check for disaggregated errors for your best model.

In [ ]:
print("Linear model")
for features in model_features:
    results = run_training_eval(model_features[features])
    print(f"Results for {features}")
    print("========================")
    print({"Train": results["train_eval"]})
    print({"Test": results["test_eval"]})

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
print("Random Forest")

for features in model_features:
    results = run_training_eval(model_features[features], model=RandomForestRegressor())
    print(f"Results for {features}")
    print("========================")
    print({"Train": results["train_eval"]})
    print({"Test": results["test_eval"]})

The best model obtained is from based on random forest using `["pm_cs", "temp", "humidity", "a_road_500", "sin_time" "cos_time", "sin_month", "cos_month"]`, i.e `model_4`

The result for our best model is:

`train RMSE= 0.85` <br>
`test RMSE= 3.46`

###### Now, let us deepen our best model

In [ ]:
results_best_model = run_training_eval(
    model_features["model_4"], model=RandomForestRegressor()
)

In [ ]:
plt.scatter(
    test_data["pm_airnow"],
    results_best_model["y_hat_test"],
    alpha=0.4,
    label="Corrected",
)
plt.scatter(
    test_data["pm_airnow"], test_data["pm_cs"], alpha=0.4, label="Not corrected"
)
plt.ylabel("AirNow")
plt.xlabel("CS")
plt.title("Test prediction from best model")
plt.legend()
plt.show()

###### Now, we should check the feature importances

In [ ]:
feature_importances = results_best_model["model"].feature_importances_

In [ ]:
fig = plt.figure(figsize=(9, 4))
plt.bar(model_features["model_4"], feature_importances)
plt.show()

The most important feature for our random forest correction model is the `PM 2.5`readings from the low-cost sensor

**Disaggregating the errors into the different locations**

In [ ]:
best_disagg_test_metric = get_disaggregated_metrics(
    test_data, results_best_model["y_hat_test"]
)
best_disagg_train_metric = get_disaggregated_metrics(
    training_data, results_best_model["y_hat_train"]
)

In [ ]:
print("best model")
print(best_disagg_test_metric["RMSE"])
print("baseline")
print(baseline_disagg_test_metrics["RMSE"])

Compare to our baseline for test data as shown below:

In [ ]:
print("best model")
print(best_disagg_train_metric["RMSE"])
print("baseline")
print(baseline_disagg_train_metrics["RMSE"])

We have lower errors in NJH and CAMP which are locations with lowers IQRs. They are both farther from the expressway than the other locations. 

Best result compared to baseline:

| Location| Baseline RMSE| Random forest RMSE|
  |---|---|---|
  |**Train**|---|---|
  |NJH | 4.36| 0.63|
  |i25_glo_1|6.67|1.01|
  |i25_glo_2|4.55|0.93|
  |i25_glo_3|5.41|0.86|
  |la_casa|6.06|0.68|
  |**Test**|---|---|
  |CAMP|2.35|1.64|
  |i25_denver|8.04|4.47|
  

Remember that the training results are obtained from data and locations that have already been seen by the model. Hence we have results that are over-optimistic for those locations. The model we have so far, however, shows usefulness in that it reduces error on test data (which are locations that have not been seen by our our model).

How do we objectively evaluate the performance of our model in locations that are in the training data?
For this, we will apply LOOO cross validation in the next notebook.